![](https://i.imgur.com/07cskyU.png)]

## El reto

El concurso es sencillo: queremos que utilices los datos de los pasajeros del Titanic (nombre, edad, precio del billete, etc.) para intentar predecir quién sobrevivirá y quién morirá.

### [Titanic - Machine Learning from Disaster](https://www.kaggle.com/c/titanic/overview)

## Los datos

### *train.csv*
* train.csv contiene los datos de un subconjunto de los pasajeros a bordo (891 pasajeros, para ser exactos -- donde cada pasajero obtiene una fila diferente en la tabla).  
* Los valores de la segunda columna ("Survived") pueden utilizarse para determinar si cada pasajero ha sobrevivido o no:
    * si es un "1", el pasajero sobrevivió.
    * si es un "0", el pasajero murió.
* Por ejemplo, el primer pasajero que aparece en train.csv es el Sr. Owen Harris Braund. Tenía 22 años cuando murió en el Titanic.

### *test.csv*
* Utilizando los patrones que encuentras en train.csv, tienes que predecir si los otros 418 pasajeros a bordo (en test.csv) sobrevivieron.
* Ten en cuenta que el archivo test.csv no tiene una columna "Survived": esta información está oculta para ti, y lo bien que lo hagas al predecir estos valores ocultos determinará tu puntuación en la competición.

### *submission.csv*
* El archivo gender_submission.csv se proporciona como un ejemplo que muestra cómo debe estructurar sus predicciones. En él se predice que todas las pasajeras sobrevivieron y que todos los pasajeros varones murieron. Sus hipótesis sobre la supervivencia serán probablemente diferentes, lo que dará lugar a un archivo de presentación diferente. Pero, al igual que este archivo, su presentación debe tener:
    * una columna "PassengerId" que contenga las identificaciones de cada pasajero de test.csv.
    * una columna "Survived" (¡que tú crearás!) con un "1" para las filas en las que crees que el pasajero ha sobrevivido, y un "0" en las que predices que el pasajero ha muerto.

In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

In [10]:
train_data = pd.read_csv("train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [11]:
test_data = pd.read_csv("test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [13]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [22]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [31]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


El código anterior calcula el porcentaje de pasajeros masculinos (en train.csv) que sobrevivieron.

De ello se desprende que casi el 75% de las mujeres a bordo sobrevivieron, mientras que sólo el 19% de los hombres vivieron para contarlo. Dado que el género parece ser un indicador tan fuerte de la supervivencia, el archivo de presentación en gender_submission.csv no es una mala primera aproximación.

Pero a fin de cuentas, esta presentación basada en el género basa sus predicciones en una sola columna. Como puede imaginarse, al considerar varias columnas, podemos descubrir patrones más complejos que pueden dar lugar a predicciones mejor informadas. Dado que es bastante difícil considerar varias columnas a la vez (o, se necesitaría mucho tiempo para considerar todos los posibles patrones en muchas columnas diferentes simultáneamente), utilizaremos el aprendizaje automático para automatizar esto por nosotros.

## Tu primer modelo de aprendizaje automático

Vamos a construir lo que se conoce como un modelo de bosque aleatorio. Este modelo se construye con varios "árboles" (en la imagen de abajo hay tres árboles, pero nosotros construiremos 100) que considerarán individualmente los datos de cada pasajero y votarán si el individuo ha sobrevivido. A continuación, el modelo de bosque aleatorio toma una decisión democrática: ¡el resultado con más votos gana!

![img](https://i.imgur.com/AC9Bq63.png)

La celda de código siguiente busca patrones en cuatro columnas diferentes ("Pclass", "Sex", "SibSp" y "Parch") de los datos. Construye los árboles del modelo de bosque aleatorio basándose en los patrones del archivo train.csv, antes de generar predicciones para los pasajeros en test.csv. El código también guarda estas nuevas predicciones en un archivo CSV submission.csv.

In [32]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
